In [1]:
import os

import numpy as np

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  # гарантируем воспроизводимость

run_env = os.getenv('RUN_ENV', 'COLLAB')
if run_env == 'COLLAB':
  from google.colab import drive
  ROOT_DIR = '/content/drive'
  drive.mount(ROOT_DIR)
  print('Google drive connected')
  root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', 'amazon_reviews')
else:
  root_data_dir = os.getenv('DATA_DIR', '/srv/data')

if not os.path.exists(root_data_dir):
  raise RuntimeError('Data dir not exists')
else:
  print('Data dir content %s: %s' % (root_data_dir, ', '.join(os.listdir(root_data_dir))))

Mounted at /content/drive
Google drive connected


RuntimeError: Data dir not exists

In [ ]:
import os
with open(os.path.join(ROOT_DIR, 'MyDrive', 'secrets', 'secrets.env'), 'r') as f:
  envs = {j[0]: j[1] for j in [i.strip().split('=') for i in f.readlines()]}
print(len(envs))

2


In [ ]:

import shutil
from IPython.display import clear_output

!pip install accelerate -U
!pip install transformers[torch]
!pip install sentence-transformers==2.2.2
# !pip install nltk==3.6.2
!pip install  backoff
!pip install  openai
clear_output()
print('Libas installed')

Libas installed


# RAG dev

[amazon reviews](https://amazon-reviews-2023.github.io/)

Plan
* rag over one dataset (healthcare)
* rag over two datasets (healthcare + electronics)

# EDA

In [ ]:
import gzip
import json

def read_raw_data(file_name, limit: int, fields = None):

  file_path = os.path.join(root_data_dir, file_name)
  res = []
  with gzip.open(file_path, 'rt') as gz_file:
      for line in gz_file:
          data = json.loads(line.strip())
          if fields is not None:
            res.append({i: j for i, j in data.items() if i in fields})
          else:
            res.append(data)
          if limit == len(res):
              break
  print('Dataset num items: %d' % len(res))
  return res

read_raw_data('Health_and_Personal_Care.jsonl.gz', limit = 3)

Dataset num items: 3


[{'rating': 4.0,
  'title': '12 mg is 12 on the periodic table people! Mg for magnesium',
  'text': 'This review is more to clarify someone else’s review bc they didn’t understand understand the labeling!  It shows 1000mg as advertised & another little label says 12mg bc 12 is on the periodic table for magnesium!  I realize not everyone takes chemistry, but 4 ppl liked his review & so misinformation is spreading.  This works. If however you are on opiate level medications that are causing constipation you should talk to your pain dr or your gastrointestinal dr & ask for a medication called Linzess which works must better & must faster, but is unnecessary for most people.  If magnesium is working for you just make sure to take it with food & drink 6-8 glasses of water per day.  Staying hydrated will really help.  Before switching to Linzess I used to take one 1,000 mg pill am & pm every day with meals & always with an 8 ounce glass of water or other liquid.',
  'images': [],
  'asin': '

TODO: sampling strategy

In [ ]:
import pandas as pd
db_sample_df = pd.json_normalize(
    read_raw_data('Health_and_Personal_Care.jsonl.gz', limit=1000, fields = None)
)
(
  db_sample_df
  .groupby('asin')['user_id']
  .count()
  .reset_index(name='popularity')
  .sort_values(by='popularity', ascending=False)
  .head()
)

Dataset num items: 1000


,asin,popularity
9,B000G2BESO,7
350,B07HSF5HTX,7
343,B07GHG9P9P,4
827,B09F7PH4QR,4
351,B07J32R15F,3


# Train embeddings

In [ ]:
def get_pytorch_model(root_dir, model_name='all-mpnet-base-v2'):
  """
  model = get_pytorch_model(root_data_dir)
  """
  from sentence_transformers import SentenceTransformer

  models_dir = os.path.join(root_dir, 'models')
  if not os.path.exists(models_dir):
      os.mkdir(models_dir)
  model_path = os.path.join(models_dir, model_name)

  if not os.path.exists(model_path):
      print('huggingface model loading...')
      embedder = SentenceTransformer(model_name)
      embedder.save(model_path)
      clear_output()
  else:
      print('pretrained model loading...')
      embedder = SentenceTransformer(model_name_or_path=model_path)
  print('model loadind done')

  return embedder

def load_corpus(db):
    corpus_texts = []
    for item in db:
      corpus_texts.append(item['text'])
    return corpus_texts

def train_embeds(corpus_texts, embedder, sentence_embedding_path, overwrite=False):
    if os.path.exists(sentence_embedding_path) and not overwrite:
        print('corpus loading from %s' % sentence_embedding_path)
        passage_embeddings = np.load(sentence_embedding_path)
    else:
        print('num rows %d' % len(corpus_texts))
        passage_embeddings = embedder.encode(corpus_texts, show_progress_bar=True)
        passage_embeddings = np.array([embedding for embedding in passage_embeddings]).astype("float32")
        with open(sentence_embedding_path, 'wb') as f:
            np.save(f, passage_embeddings)
        print('corpus saved to %s' % sentence_embedding_path)
    print('Num embeddings %d' % passage_embeddings.shape[0])
    return passage_embeddings

model = get_pytorch_model(root_data_dir)
db = read_raw_data(
    'Health_and_Personal_Care.jsonl.gz',
    limit = -1, fields = ['rating', 'text', 'title', 'asin']
)
corpus = load_corpus(db)
data_version = 0
embeds = train_embeds(
    corpus, model,
    os.path.join(root_data_dir, f'corpus_embeds_{data_version}.npy'),
    overwrite=False
)

pretrained model loading...
model loadind done
Dataset num items: 494121
corpus loading from /content/drive/MyDrive/sweed_data/corpus_embeds_0.npy
Num embeddings 494121


In [ ]:
import datetime
import hashlib

import backoff
import openai
from openai import OpenAI


client = OpenAI(
    api_key=envs["OPENAI_API_KEY"]
)


@backoff.on_exception(backoff.expo, openai.APIError)
@backoff.on_exception(backoff.expo, openai.RateLimitError)
@backoff.on_exception(backoff.expo,openai.Timeout)
@backoff.on_exception(backoff.expo, RuntimeError)
def gpt_query(gpt_params, verbose: bool = False, avoid_fuckup: bool = False) -> dict:
    print('connecting OpenAI...')
    if verbose:
        print(gpt_params["messages"][1]["content"])
    response = client.chat.completions.create(
        **gpt_params
    )
    gpt_response = response.choices[0].message.content
    if avoid_fuckup:
        if '[' in gpt_response or '?' in gpt_response or '{' in gpt_response:
            raise RuntimeError
    res = {'recs': gpt_response}
    res.update({'prompt_tokens': response.usage.completion_tokens, 'prompt_tokens': response.usage.prompt_tokens, 'total_tokens': response.usage.total_tokens})
    seed_phrase = f'{str(datetime.datetime.now().timestamp())}{gpt_response}'
    schedule_id = str(hashlib.md5(seed_phrase.encode('utf-8')).hexdigest())[:12]
    res.update({'id': schedule_id})
    return res

def gen_candadates(db, ids):
  candidates = '\n'.join(["item_id: %s; review: %s" % (db[i]['asin'], db[i]['text']) for i in ids])
  return candidates

def generate(db, ids, verbose=False):
    gpt_params = {
        'model': 'gpt-3.5-turbo',
        'max_tokens': 500,
        'temperature': 0.7,
        'top_p': 0.5,
        'frequency_penalty': 0.5,
    }
    gpt_promt = promt_generation(gen_candadates(db, ids))
    if verbose:
        print(gpt_promt)
    messages = [
        {
          "role": "system",
          "content": "You are a helpful assistant for medicine shopping",
        },
        {
          "role": "user",
          "content": gpt_promt,
        },
    ]
    gpt_params.update({'messages': messages})
    res = gpt_query(gpt_params, verbose=False)

    return res

def top_similar(query_embed, candidates_embeds, top=10):
  from sklearn.metrics.pairwise import cosine_similarity

  sims = cosine_similarity(query_embed.reshape(1, -1), candidates_embeds)[0]
  top_similar_idx = [int(i) for i in np.argsort(-np.abs(sims))][:top]
  return top_similar_idx

def retrieve(query):
  query_embed = model.encode(query, show_progress_bar=False)
  indexes = top_similar(query_embed, embeds)
  return indexes

def get_retrieved_content(db, ids):
  res = [{db[i]['asin']: db[i]['text']} for i in ids]
  return res

def promt_generation(candidates):
  # TODO: use jinja2
  promt = f"""
      Next rows below is an item_id reviews.
      {candidates}
      Utilize reviews to determine the best item_id.
      Avoid including actual reviews; rephrase them succinctly.
      Keep the recommendation under 50 words. Avoid starting with "Based on reviews"; opt for a more creative approach!
      Recommendation:
  """
  return promt

query = "headache tablets"
ids = retrieve(query)
get_retrieved_content(db, ids)

[{'B002C6467S': 'Best headache pills ever'},
 {'B000V0D0QK': "Best headache product. I keep one at work and one in my purse. Works much faster than pills. I also like their 'sinus help'."},
 {'B017MV8V64': "Terrible product. Complete waste of money... Save yourself the headache and don't ever spend money for these pills. They do nothing."},
 {'B00JQSW4MY': 'My only non drug solution for a bad headache...works every time...'},
 {'B01M04LW7Z': 'Only medicine that relieves my tension headaches.'},
 {'B002C6467S': 'best headache medicine ever'},
 {'B002C6467S': 'Best headache medicine'},
 {'B079YWWJZQ': 'I have to be careful with these or they give me a terrific headache. Be careful of taking with anything else.'},
 {'B002C6467S': 'This medicine works great on my headache.'},
 {'B002C6467S': "This is the only medicine that cures my headaches it's the best"}]

In [ ]:

res = generate(db, ids)
print(res['recs'].replace('. ', '.\n'))

connecting OpenAI...
I recommend item_id B002C6467S for effective headache relief based on positive user experiences.
This medicine has consistently been praised for its fast and reliable results, making it a top choice for those seeking quick and lasting relief from headaches.


In [ ]:
query = "getting cold"
ids = retrieve(query)
res = generate(db, ids)
print(res['recs'].replace('. ', '.\n'))

connecting OpenAI...
For long-lasting warmth and coverage over a large area, consider item_id B06XJ253CH.
It effectively stays cold and provides ample warmth without needing to be reheated frequently.


In [ ]:
query = "bad cough"
ids = retrieve(query)
res = generate(db, ids)
print(res['recs'].replace('. ', '.\n'))

connecting OpenAI...
For effective relief from cough, consider item_id B009RPA6RQ as customers have experienced quick results with just one dose.
It's a convenient solution to help alleviate cough symptoms.


In [ ]:
get_retrieved_content(db, ids)

[{'B002ZXMXD2': 'Slows cough'},
 {'B013CX7R5M': 'Got rid of my cough'},
 {'B0009RO11I': 'Stops coughing.'},
 {'B009RPA6RQ': 'Had a cough. Took 1 dose and the cough was gone.'},
 {'B01IAI712S': 'Excellent for a cough!'},
 {'B000G2BESO': 'My husband is coughing MUCH less I have one by his chair and one by head of the bed'},
 {'B07177YRWD': 'This is a trip to the Dr saver!!  Colds happen, but sometimes they are worse than a cough - and instead of waiting to see if it gets better, just blow your nose!  Product is amazing!'},
 {'B01867RCR2': 'Stinks smells like a pine tree'},
 {'B01MA3FA6H': 'WE GOT THIS FOR COUGH AND WHEN WE USED IT ITS WORK'},
 {'B08V8G6M5F': 'Bad knee'}]

In [ ]:
query = "insomnia"
ids = retrieve(query)
res = generate(db, ids)
print(res['recs'].replace('. ', '.\n'))

connecting OpenAI...
Recommendation: Embrace restful nights and bid farewell to insomnia with the item_id B00RY8HUOK, which has helped many users overcome sleepless nights and reclaim their peace of mind.


In [ ]:
get_retrieved_content(db, ids)

[{'B0986QSMJJ': 'Sleeping'},
 {'B085RFQVNC': 'Sleep'},
 {'B01COI6EHI': 'Sleep'},
 {'B00ZTC56O4': 'Sleep'},
 {'B07FSCW71V': 'Sleep'},
 {'B00A3Z44RW': 'Sleep'},
 {'B00RY8HUOK': 'I have suffered with horrible insomnia so bad it caused me to take early retirement. This preparation, along with an antidepressant, has defeated my horrible foe.'},
 {'B085RFQVNC': 'Day sleeping.'},
 {'B06XXCV77N': 'sleeping well'},
 {'B085X863KB': 'Insomnia is a very headache. Long-term insomnia has bothered me. Since I bought this product, spraying on the pillow every time before going to bed can make the whole person relax and go to sleep quickly.'}]

In [ ]:
query = "depression"
ids = retrieve(query)
res = generate(db, ids)
print(res['recs'].replace('. ', '.\n'))

connecting OpenAI...
After analyzing the feedback provided, I suggest considering item_id B01LXSDEV4 for its positive association with relaxation.
This item has garnered favorable reviews related to promoting a sense of calm and tranquility.
It could be a beneficial choice for your needs.


In [ ]:
get_retrieved_content(db, ids)

[{'B07WR9NQPM': 'trying for depression'},
 {'B01LXSDEV4': 'Relaxation.'},
 {'B08HNFBDTK': 'Relaxation'},
 {'B06XNLLR6C': 'Relaxation'},
 {'B076H8Z7P7': 'Medicine'},
 {'B0083I38EQ': 'Meds'},
 {'B01LZLSJ7B': 'Health'},
 {'B093TMDWPZ': 'Health'},
 {'B006T4KZH4': 'Health'},
 {'B099682GNL': 'Health'}]